In [5]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

Generate clean sentences

In [6]:
def read_article(file_name):
  file = open(file_name, "r")
  filedata = file.readlines()
  article = filedata[0].split(". ")
  sentences = []

  for sentence in article:
    print(sentence)
    sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

  return sentences

Similarity between the sentences

In [9]:
def sentence_similarity(sent1, sent2, stopwords=None):
  if stopwords is None:
    stopwords = []


  sent1 = [w.lower() for w in sent1]
  sent2 = [w.lower() for w in sent2]

  all_words = list(set(sent1 + sent2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  #build the vector for the first sentence
  for w in sent1:
    if w in stopwords:
      continue
    vector1[all_words.index(w)] += 1

  #build the vector for the second sentence
  for w in sent2:
    if w in stopwords:
      continue
    vector2[all_words.index(w)] += 1

Generating the similarity matrix

In [10]:
def build_similarity_matrix(sentences, stop_words):
  #create an empty similarity matrix
  similarity_matrix = np.zeros((len(sentences), len(sentences)))

  for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
      if idx1 == idx2:
        continue
      similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2])

  return similarity_matrix

Generate summary method

In [29]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')
nltk.download('punkt')


def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.read()
    sentences = nltk.sent_tokenize(filedata)
    sentences = [sentence.replace("[^a-zA-Z]", " ").split(" ") for sentence in sentences]
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, output_file, top_n=2):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

      # Step 5 - Output the summarized text to a new file
    with open(output_file, "w") as summary_file:
        summary_file.write(". ".join(summarize_text))

    print(f"Summary has been saved in '{output_file}'")

# let's begin
generate_summary( "new.txt","new_summary1.txt", 2)

Indexes of top ranked_sentence order are  [(0.0759334791499862, ['The', 'paper', 'narrows', 'the', 'scope', 'to', 'the', 'issue', 'of', 'drug', 'abuse', 'and', 'addiction', 'since', 'this', 'is', 'arguably', 'the', 'basic', 'source', 'of', 'information', 'for', 'the', 'other', 'forms', 'of', 'addictions', 'as', 'well.']), (0.06169697254346787, ['Thesis', 'Statement\n“The', 'affects', 'of', 'drug', 'addictions', 'and', 'its', 'influence', 'on', 'other', 'forms', 'of', 'everyday', 'addictions”', 'This', 'paper', 'is', 'an', 'analysis', 'of', 'the', 'effect', 'of', 'everyday', 'addictions', 'its', 'effects', 'and', 'possibilities', 'as', 'well', 'as', 'implications', 'over', 'future', 'endeavours.']), (0.05405318381129114, ['Arguably', 'the', 'other', 'forms', 'of', 'addictions', 'are', 'often', 'side', 'effects', 'of', 'drug', 'abuse', 'and', 'addiction.']), (0.0483482436746138, ['The', 'paper', 'also', 'forms', 'an', 'analysis', 'over', 'issue', 'of', 'other', 'probable', 'cause', 'in',

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
